## Data Collection

To build a model, I will need a large number of plot summaries. I have decided to use
Wikipedia as a source for these summaries, since they have lengthy summaries, and they have
lists of movies that make it easy to search.

To begin with I am testing out individual queries to Wikipedia, to make sure I can pull
multiple entries from a list.

In [ ]:
import requests

S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

TITLE = "Category:1961 films|Category:1965 films"

PARAMS = {
    'action': "query",
    'list': 'categorymembers',
    'cmtitle': TITLE,
    'cmlimit': '10',
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

In [ ]:
print(DATA['error'])
#[item['title'] for item in DATA['query']['categorymembers']]

In [121]:
years = []
URL = "https://en.wikipedia.org/w/api.php"

for y in range(1960,1965):
    TITLE = "Category:" + str(y) + " films"

    PARAMS = {
        'action': "query",
        'list': 'categorymembers',
        'cmtitle': TITLE,
        'cmlimit': '10',
        'format': "json",
    }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    ids = [item['pageid'] for ind,item in enumerate(DATA['query']['categorymembers']) if ind>1]
    years.append(ids)
years

In [122]:
len(sci_fi)

600

In [ ]:
DATA['parse']

It is difficult to extract plaintext from the page content that the standard api returns,
so I am testing out some wrappers for the api, which may provide extra functionality.

In [ ]:
from mediawiki import MediaWiki
import wikipedia

query = "Category:1961 films"

c = wikipedia.page(pageid=years[3][6])
c.title

In [ ]:
fulltext = c.content

In [ ]:
import string
ind1 = fulltext.find('== Plot ==') + 10
ind2 = fulltext.find('==',ind1)
plottext = fulltext[ind1:ind2]
plottext.replace('\n',' ').translate(str.maketrans('','',string.punctuation))

The `wikipedia` package automatically returns a field called `content` that contains the page as plaintext,
so I am going to use that for the specific pages to make extracting the plot easier.

In [44]:
import datetime
import time
from wikiparse_movies import WikiParser

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


I have created a WikiParser object with all the functionality I need to make calls to the wikipedia api, and now I am running it to get plots from the years 1960-1965.

In [ ]:
now = datetime.datetime.now()

wp = WikiParser()
yrs_1960_1970 = wp.get_years(1960,1965)

later = datetime.datetime.now()
elapsed = later-now
print("Time: ", elapsed) 
#year_1960 = movies
print(len(yrs_1960_1970))

In [ ]:
import pandas as pd

yrs1_df = pd.DataFrame.from_dict(yrs_1960_1970)
yrs1_df.head()

In [ ]:
yrs1_df = yrs1_df[~yrs1_df['title'].str.startswith('List')]

After some examination, there are a few problems with this data. The language used to describe foreign films it somehwat unique, and tends to trow off the topic modeling, so it is necessesary to refine the dataset by only using english-language films. Fortunately this is a specific category on wikipedia, so in lieu of search by each year, I only need to search for english-language films. 

In [64]:
#430
#

now = datetime.datetime.now()

wp = WikiParser()
english_films_2 = wp.get_plots_from_year('English-language',start=431,skip=True)

later = datetime.datetime.now()
elapsed = later-now
print("Time: ", elapsed) 
#year_1960 = movies
print(len(english_films_2))

page: 431 parsing... . . . . . . . . .  
page: 432 parsing... . . . . . . . . .  
page: 433 parsing... . . . . . . . . .  
page: 434 parsing... . . . . . . . . .  
page: 435 parsing... . . . . . . . . .  
page: 436 parsing... . . . . . . . . .  
page: 437 parsing... . . . . . . . . .  
page: 438 parsing... . . . . . . . . .  
page: 439 parsing... . . . . . . . . .  
page: 440 parsing... . . . . . . . . .  
page: 441 parsing... . . . . . . . . .  
page: 442 parsing... . . . . . . . . .  
page: 443 parsing... . . . . . . . . .  
page: 444 parsing... . . . . . . . . .  
page: 445 parsing... . . . . . . . . .  
page: 446 parsing... . . . . . . . . .  
page: 447 parsing... . . . . . . . . .  
page: 448 parsing... . . . . . . . . .  
page: 449 parsing... . . . . . . . . .  
page: 450 parsing... . . . . . . . . .  
page: 451 parsing... . . . . . . . . .  
page: 452 parsing... . . . . . . . . .  
page: 453 parsing... . . . . . . . . .  
page: 454 parsing... . . . . . . . . .  
page: 455 parsin

In [65]:
yrs4_df = pd.DataFrame(english_films_2)
yrs4_df.head()

,summary,title
0,nn and Raggedy Andy is a tworeel cartoon produ...,Raggedy Ann and Raggedy Andy (1941 film)
1,Nita a divorced mother of two boys works as a...,Raggedy Man
2,The film centres around the character of Tom ...,The Raggedy Rawney
3,In 1964 an aging overweight Italian American ...,Raging Bull
4,Bruce Pritchard Malcolm McDowell is a 24yearo...,The Raging Moon


In [66]:
yrs4_df.shape

(15821, 2)

Now that I have the dataset compiled, I will save it as a json file for temporary storage, until I can get it uploaded to a MongoDB.

In [67]:
yrs4_df.to_json('data/eng_431_601.json')

In [ ]:
movies1_df = pd.read_json('1960_1964.json')
movies2_df = pd.read_json('1965_1970.json')
movies_df = movies1_df.append(movies2_df)
print(movies_df.shape)
movies_df.head()

In [95]:
import pickle
with open('samples.pkl','rb') as f:
    data = pickle.load(f)
m_corpus = data[0]
m_id2word = data[1]
sample = data[2]
sample1 = data[3]
sample2 = data[4]

In [106]:
def get_tag(idx,doc,st):
    if idx % 1000 == 0:
        print(idx)
    return st.tag(doc.split())

def tag_words(texts, st):
    tags = [get_tag(idx,doc,st) for idx,doc in enumerate(texts)]
    print(tags[:10])
    names = [set([word for word,tag in doc if tag != 'O']) for doc in tags]
    return set.union(*names)


In [107]:
st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz',
                       '/Users/alexanderbailey/Documents/stanford-ner-2018-10-16/stanford-ner.jar')
names2 = tag_words(sample2, st)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
[[('rmenian', 'O'), ('Դանիէլ', 'O'), ('is', 'O'), ('a', 'O'), ('multi', 'O'), ('award', 'O'), ('winning', 'O'), ('arthouse', 'O'), ('short', 'O'), ('film', 'O'), ('by', 'O'), ('British', 'O'), ('writer', 'O'), ('and', 'O'), ('director', 'O'), ('Garo', 'PERSON'), ('Berberian', 'PERSON'), ('telling', 'O'), ('the', 'O'), ('story', 'O'), ('of', 'O'), ('the', 'O'), ('last', 'O'), ('months', 'O'), ('of', 'O'), ('poet', 'O'), ('Taniel', 'PERSON'), ('Varoujan', 'PERSON'), ('until', 'O'), ('his', 'O'), ('murder', 'O'), ('during', 'O'), ('the', 'O'), ('Armenian', 'O'), ('Genocide', 'O'), ('at', 'O'), ('the', 'O'), ('age', 'O'), ('of', 'O'), ('31', 'O'), ('the', 'O'), ('day', 'O'), ('of', 'O'), ('his', 'O'), ('son’s', 'O'), ('birth', 'O'), ('The', 'O'), ('film', 'O'), ('is', 'O'), ('the', 'O'), ('first', 'O'), ('to', 'O'), ('deal', 'O'), ('with', 'O'), ('the', 'O'), ('story', 'O'), ('of', 'O'), ('a', 'O'), ('man', 'O'), ('considered

In [108]:
import pickle
with open('data/names_list2.pkl', 'wb') as f:
    pickle.dump(names2, f)

In [129]:
movies1_df = pd.read_json('data/eng_1_430.json')
movies2_df = pd.read_json('data/eng_431_601.json')
movies = movies1_df.append(movies2_df)
print(movies.shape)
movies.head()

(56005, 2)


,summary,title
0,nown as 1 Life On The Limit is a 2013 document...,1 (2013 film)
1,While on the phone with his girlfriend Jill w...,+1 (film)
10,Gully Mercer Mick Rossi leads a group of prol...,2:22 (2008 film)
100,o Christmas is a Ghanaian drama movie about Re...,6 Hours To Christmas
1000,r is a 1995 American romantic drama television...,The Affair (1995 film)


## Storing with MongoDB

Now that I have the data backed up locally, I will attempt to store it remotely with Mongo.

In [130]:
from pymongo import MongoClient
from pprint import pprint
import json

with open('/Users/alexanderbailey/.secret/mongo_creds.json','r') as f:
    params = json.load(f)

url = 'mongodb+srv://zmbailey:' + params['password'] + '@cluster0-ykzgc.mongodb.net/test?retryWrites=true&w=majority'

client = MongoClient(url)
db=client.admin

serverStatusResult=db.command("serverStatus")
pprint(serverStatusResult)

{'$clusterTime': {'clusterTime': Timestamp(1562004936, 1),
                  'signature': {'hash': b'\x85Y\xca\x85\xbf\xa3\xb9\xa8'
                                        b'\x8c\x08\n\xe60 \x0fP\xe6vC\x7f',
                                'keyId': 6703621046206988289}},
 'asserts': {'msg': 0, 'regular': 0, 'rollovers': 0, 'user': 0, 'warning': 0},
 'connections': {'available': 98, 'current': 2, 'totalCreated': 26},
 'extra_info': {'note': 'fields vary by platform', 'page_faults': 0},
 'host': 'cluster0-shard-00-01-ykzgc.mongodb.net:27017',
 'localTime': datetime.datetime(2019, 7, 1, 18, 15, 38, 943000),
 'mem': {'bits': 64,
         'mapped': 0,
         'mappedWithJournal': 0,
         'resident': 0,
         'supported': True,
         'virtual': 0},
 'metrics': {'atlas': {'bytesInWrites': 0,
                       'connectionPool': {'totalCreated': 278}}},
 'network': {'bytesIn': 35770, 'bytesOut': 511897, 'numRequests': 188},
 'ok': 1.0,
 'opcounters': {'command': 183,
           

In [131]:
mongodb = client.movieplots

In [134]:
#mon = mongodb.movies.insert_many(movies.to_dict('records'))
print(mon.inserted_ids[:10])

[ObjectId('5d1a4e861293e4051fb185ae'), ObjectId('5d1a4e861293e4051fb185af'), ObjectId('5d1a4e861293e4051fb185b0'), ObjectId('5d1a4e861293e4051fb185b1'), ObjectId('5d1a4e861293e4051fb185b2'), ObjectId('5d1a4e861293e4051fb185b3'), ObjectId('5d1a4e861293e4051fb185b4'), ObjectId('5d1a4e861293e4051fb185b5'), ObjectId('5d1a4e861293e4051fb185b6'), ObjectId('5d1a4e861293e4051fb185b7')]
